# Full Simulated Tool Calling Example (Back-and-Forth)

This notebook demonstrates a full, end-to-end example of **simulated tool calling** using an OpenAI-compatible API (such as Featherless), *without* native tool calling support.

The LLM suggests a tool call by outputting a JSON object, the code executes the tool, and the result is provided back to the LLM for a final answer.

This pattern is inspired by the [OpenRouter tool calling documentation](https://openrouter.ai/docs/features/tool-calling), but uses prompt-based simulated tool calls instead of the OpenAI `tools` parameter.

## Prerequisites
1. Sign up for an account at [Featherless](https://featherless.ai/register)
2. Subscribe to a plan and get your API key from [API Keys](https://featherless.ai/account/api-keys)

## Setup

In [1]:
# !pip install openai requests

In [2]:
import json
import requests
from openai import OpenAI

client = OpenAI(
    base_url="https://api.featherless.ai/v1",
    api_key="YOUR FEATHERLESS API KEY",
)

def search_gutenberg_books(search_terms):
    search_query = " ".join(search_terms)
    url = "https://gutendex.com/books"
    response = requests.get(url, params={"search": search_query})
    simplified_results = []
    for book in response.json().get("results", []):
        simplified_results.append({
            "id": book.get("id"),
            "title": book.get("title"),
            "authors": book.get("authors"),
        })
    return simplified_results

TOOL_MAPPING = {
    "search_gutenberg_books": search_gutenberg_books
}

## Step 1: User asks a question

We'll use the following task:

> What are the titles of some James Joyce books?

In [3]:
task = "What are the titles of some James Joyce books?"

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": task},
]

## Step 2: Simulate a tool call (model outputs a JSON object)

We instruct the model to output a JSON object describing the tool call it wants to make. We use `response_format={"type": "json_object"}` to ensure valid JSON.

In [4]:
tool_call_prompt = (
    "You have access to a function: search_gutenberg_books(search_terms: list of str).\n"
    "When asked about books, respond ONLY in the following JSON format:\n"
    '{\n  "function": "search_gutenberg_books",\n  "arguments": {"search_terms": ["<term1>", "<term2>"]}\n}\n'
    "Do not answer the question directly. Only output the JSON."
)

messages_tool = [
    {"role": "system", "content": tool_call_prompt},
    {"role": "user", "content": task},
]

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=messages_tool,
    response_format={"type": "json_object"},
)
tool_call_json = json.loads(response.choices[0].message.content)
print("Model tool call:", tool_call_json)

Model tool call: {'function': 'search_gutenberg_books', 'arguments': {'search_terms': ['James Joyce']}}


## Step 3: Execute the tool call and append the result as a message

We parse the tool call, execute the function, and append the result as a `tool` message.

In [5]:
tool_name = tool_call_json["function"]
tool_args = tool_call_json["arguments"]
tool_result = TOOL_MAPPING[tool_name](**tool_args)

messages_with_tool = [
    {"role": "system", "content": tool_call_prompt},
    {"role": "user", "content": task},
    {"role": "tool", "name": tool_name, "content": json.dumps(tool_result)},
]
print("Tool result (truncated):", tool_result[:2])  # Show just a couple of results

Tool result (truncated): [{'id': 4300, 'title': 'Ulysses', 'authors': [{'name': 'Joyce, James', 'birth_year': 1882, 'death_year': 1941}]}, {'id': 2814, 'title': 'Dubliners', 'authors': [{'name': 'Joyce, James', 'birth_year': 1882, 'death_year': 1941}]}]


## Step 4: Ask the model to answer the original question using the tool result

We now prompt the model to use the tool result to answer the user's question.

In [6]:
answer_prompt = (
    "You are a helpful assistant.\n"
    "You have access to the results of a function call.\n"
    "Use the tool result to answer the user's original question."
)

messages_final = [
    {"role": "system", "content": answer_prompt},
    {"role": "user", "content": task},
    {"role": "tool", "name": tool_name, "content": json.dumps(tool_result)},
]

response_final = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=messages_final,
)
print("\nFinal answer:\n", response_final.choices[0].message.content)


Final answer:
 The titles of some James Joyce books are:

1. Ulysses
2. Dubliners
3. A Portrait of the Artist as a Young Man
4. Index of the Project Gutenberg Works of James Joyce
5. Chamber Music
6. Exiles: A Play in Three Acts


---
## Summary

- The LLM suggests a tool call by outputting a JSON object (simulated tool calling)
- The code executes the tool and appends the result as a `tool` message
- The LLM uses the tool result to answer the user's original question

This pattern works with any OpenAI-compatible endpoint, even if it does not support native tool calling.